# NFT Wash Trading

Here we can build a lab notebook cell by cell.

The first task is to get us both on the same page on what data we'll be using for the project, and then we can work on loading it in and working with it.

## Data import

In [4]:
##Data import code here.
import os
import pandas as pd
cwd = os.getcwd()

In [5]:
import requests

payload = {'module':'account', 
    'action':'tokentx', 
    'address':'0x40b39bacd1658fffa015468ba88303f3d67a8740',
    'startblock':'0',
    'endblock':'99999999',
    'page':'0',
    'offset':'0',
    'sort':'asc',
    'apikey':'H2V5PTEJQ35UGTIXBZHW5SR37ID9HG2RKR'
    }
r = requests.get('https://api.etherscan.io/api', params=payload)

r.json()['result']

[{'blockNumber': '13695427',
  'timeStamp': '1638006989',
  'hash': '0xfac1e1e2951786d00bef2cb68cd20c7c9b0b335c8bcd71fabb5d7631c413d0af',
  'nonce': '12',
  'blockHash': '0x35c390c40e49d51788e01f0c8d8f355295826ac81fbfebcb489b28425565ca81',
  'from': '0x0000000000000000000000000000000000000000',
  'contractAddress': '0x1b7cb8fc659ec17c4bc1e7cdff13022308885437',
  'to': '0x40b39bacd1658fffa015468ba88303f3d67a8740',
  'value': '24017268518518518518',
  'tokenName': 'NoundlesRainbows',
  'tokenSymbol': 'Rainbows',
  'tokenDecimal': '18',
  'transactionIndex': '136',
  'gas': '225000',
  'gasPrice': '69394738995',
  'gasUsed': '74866',
  'cumulativeGasUsed': '10500253',
  'input': 'deprecated',
  'confirmations': '788193'},
 {'blockNumber': '13701435',
  'timeStamp': '1638089549',
  'hash': '0x5087e43cf460d1a6f372d4c8d4185bd9f577f681091c8e50925b07dcc7d91e5d',
  'nonce': '104',
  'blockHash': '0x89af84315dbba94bfb1fcf473f90ad65ae7f5e8f82e5d6b968865bdeedb94eb4',
  'from': '0x00000000000000000

In [10]:
#List of csv collections
collectionCSVs = [
    "0n1_force.csv",
    "axie_infinity.csv",
    "azuki.csv",
    "bored_ape.csv",
    "clone_x.csv",
    "coolmonkes.csv",
    "creature_world.csv",
    "creepz_reptile.csv",
    "creeps.csv",
    "cryptoadz.csv",
    "cryptobatz.csv",
    "cryptokitties.csv",
    "cryptopunks.csv",
    "cryptoskulls.csv",
    "cyberkongz_vx.csv",
    "DeadFellaz.csv",
    "decentraland_wearables.csv",
    "doge_pound.csv",
    "doodles.csv",
    "dr_ETHvil.csv",
    "emblem_vaul.csv",
    "FLUF_world_thingies.csv",
    "fomo_mofos.csv",
    "full_send.csv",
    "hape_prime.csv",
    "hashmasks.csv",
    "lil_heroes.csv",
    "lostpoets.csv",
    "meebits.csv",
    "mekaverse.csv",
    "metroverse.csv",
    "mutant_ape.csv",
    "my_curio_cards.csv",
    "phantabear.csv",
    "pudgypenguins.csv",
    "punkcomics.csv",
    "rarible.csv",
    "rtfkt.csv",
    "sorare.csv",
    "superrare.csv",
    "wolf_game.csv",
    "world of women.csv",
    "wvrps.csv",
    "x_rabbits.csv"
]

In [12]:
import requests
import pickle
readIn = ['seller_address']

#Change range depending on whether you are getting data
#for first or second half of list
for i in range(1,21):
    #Load in the csv
    bored_ape_location = cwd + "/data/" + collectionCSVs[i]
    bored_ape = pd.read_csv(bored_ape_location, usecols = readIn)

    #Infers some column data types to float and int to save some space
    bored_ape_converted = bored_ape.infer_objects()

    #Delete rows without wallet address
    bored_ape_converted.dropna(subset=['seller_address'], inplace=True)

    #Usernames that are listed as NaN are unnamed on OpenSea but still have a wallet attached.

    #Create a dict to store all the transactions
    all_wallet_dict = {}
    count = 1

    for seller in bored_ape_converted['seller_address']:
        
        wallet_dict = {}
        address = seller
        print("Row: " + str(count - 1) + " out of " + str(len(bored_ape_converted.index)))
        
        #Build a simple request on etherscan to scale up for each seller wallet in opensea data
        payload = {'module':'account', 
                   'action':'tokentx', 
                   'address':address,
                   'startblock':'0',
                   'endblock':'99999999',
                   'page':'0',
                   'offset':'0',
                   'sort':'asc',
                   'apikey':'H2V5PTEJQ35UGTIXBZHW5SR37ID9HG2RKR'
                  }

        try:
            r = requests.get('https://api.etherscan.io/api', params=payload)
            
            for transaction in r.json()['result']:
                if transaction['from'] == str(address):
                    wallet_dict[transaction['to']] = 'sent'
                else:
                    wallet_dict[transaction['from']] = 'recieved'
            all_wallet_dict[address] = wallet_dict
            
        except:
            print("===== ERROR ON ROW " + str(count -1) + " =====")
            errorFile, sep, tail = collectionCSVs[i].partition('.')
            errorFile = 'errors/' + errorFile + '_ErrorRows.txt'
            with open(errorFile, 'a+') as f:
                f.write('\n' + str(count - 1))

        finally:
            count = count + 1

    #Write collection dictionary to pickle file
    outFile, sep, tail = collectionCSVs[i].partition('.')
    outFile = outFile + '.pkl'
    with open(outFile, 'wb+') as f:
        pickle.dump(all_wallet_dict, f)
    print("File saved: " + outFile)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\19044\\Documents\\Senior Project\\NFT-Wash-Analysis/data/0n1_force.csv'

In [7]:
#Shows the result, a dict with the key being the wallet address
#and the value being all transacations + type
print(all_wallet_dict)

{'0x8323cc95c6fc88c832086e38869cfe1d834a4980': {'0xc3fbc3f485f0d9b0bd21b13a4aaa8340160156cb': 'recieved', '0xa5409ec958c83c3f309868babaca7c86dcb077c1': 'sent', '0xba30e5f9bb24caa003e9f2f0497ad287fdf95623': 'sent', '0xedd3ee98fd6050d104ccf983cd13948557057a8f': 'sent', '0xd5f76a01d4ebe570a9252f1a8459b44fe8f4b5bd': 'sent', '0xfc007068c862e69213dc7aa817063b1803d4e941': 'sent', '0x45d8f7db9b437efbc74ba6a945a81aaf62dceda7': 'sent', '0x1b829b926a14634d36625e60165c0770c09d02b2': 'sent', '0xc2c747e0f7004f9e8817db2ca4997657a7746928': 'sent', '0x7be8076f4ea4a4ad08075c2508e481d6c946d12b': 'sent', '0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d': 'sent', '0x1b811fab3618e727d5d38d7e6338262ca372a3ca': 'sent', '0xfcc617a026bc7614db0debc00e72ea37d9ab9317': 'sent', '0x35a5add062f5d164bfecc8b5f2e5da10a09872e9': 'sent', '0xcde1efe3c576c900a257687aa95b2efad7150227': 'sent', '0x91bbc6dcfef7a30505106bf887528218cbda3f83': 'sent', '0xdbc410bd7a9a83c3cbe501a26cabada4a60826e2': 'sent', '0xb80a6253ebbdd2f72fe5b6fd0f8

In [8]:
all_wallet_list = []
for wallet_id in bored_ape_converted['winner_account_address']:
    all_wallet_list.append(wallet_id)
#Remove duplicates
all_wallet_list = set(all_wallet_list)
#all_wallet_list

# Dont know what the first nan value is but heres where I left off for the day.

## Analysis

I'll look at the paper again and determine a similar structure of analysis. Market by market, method by method.

In [9]:
##Analysis code here

## Results

In [10]:
## Hopefully some interesting results here